In [177]:
import csv
import os
import pandas as pd

In [178]:
import twitter
api = twitter.Api(consumer_key='yylWBvKsFiT6TVutxDtw7gW9W',
                  consumer_secret='Z3tdgCzqMn5YYC38xzhShpMQVcH340WUzevRANmBGM0Di8rDfs',
                  access_token_key='82013415-pX5ss0oBh37aFm3jgBI0JxSls1DVIltroLZl0PgdI',
                  access_token_secret='UdkJwLWkT5a5Bk3Ej5eTqlOuBmtU8WV0DeekLQWwDSmXo',
                  sleep_on_rate_limit=True)

In [217]:
api.GetStatus(135950336207761409)

Status(ID=135950336207761409, ScreenName=bree_lanee, Created=Mon Nov 14 05:21:00 +0000 2011, Text="@topmodel_29 HOLD UP flag on the play...i feel some type of way.I've called you twice, once on your #, left 2 vm and you tweeting smh #hurt")

In [188]:
s = api.GetStatus(135724218678657024).text
s.strip('\n')

df = pd.DataFrame({"s":[s.strip('\n')]})
df.to_csv("../data/test_to_csv.csv")

In [166]:
tw_df = pd.read_csv("../data/raw/dev.txt", delimiter='\t', names=['t_id', 'emotion'])
tw_df.shape

(247798, 2)

In [176]:
import numpy as np
from multiprocessing import cpu_count, Pool
 
cores = cpu_count() # Number of CPU cores on your system
partitions = cores # Define as many partitions as you want
batch_num = 10000 # The size of api.GetStatus input list
 
def parallelize(df, func):
    df_split = np.array_split(df, partitions)
    pool = Pool(cores)
    data = np.concatenate(pool.map(func, df_split))
    pool.close()
    pool.join()
    return data.tolist()


def get_statuses(df):
    try:
        return np.asarray(api.GetStatuses(df['t_id']))
    except twitter.error.TwitterError as e:
        print(e)
        return []

    
def parse_status(s_list):
    tw_list = [(s.id, s.text) for s in s_list]
    tw_df = pd.DataFrame.from_records(tw_list, columns=["t_id", "status"])
    return tw_df
    
    
def load_tweets(in_fname, out_fname):
    print("Load " + in_fname)
    tw_df = pd.read_csv(in_fname, delimiter='\t', names=['t_id', 'emotion'])
    record_num = tw_df.shape[0]
    epoch_size = int(record_num / (batch_num * cores))

    out_fd = open(out_fname, 'a')
    for i in range(0, epoch_size):
        print("Processing %d - %d" % (i*batch_num*cores, (i+1)*batch_num*cores))
        if (i+1)*batch_num*cores < record_num:
            parse_status(parallelize(tw_df[i*batch_num*cores: (i+1)*batch_num*cores], 
                                     get_statuses)).to_csv(out_fd, header=False)
        else:
            parse_status(parallelize(tw_df[i*batch_num*cores:], 
                                     get_statuses)).to_csv(out_fd, header=False)
    out_fd.close()
    print("Done")

In [175]:
file_list = [
    "dev.txt",
    "test.txt",
    "train_1.txt",
    "train_2_10.txt",
    "train_2_1.txt",
    "train_2_2.txt",
    "train_2_3.txt",
    "train_2_4.txt",
    "train_2_5.txt",
    "train_2_6.txt",
    "train_2_7.txt",
    "train_2_8.txt",
    "train_2_9.txt"
]

for f in file_list:
    load_tweets("../data/raw/" + f, "/mnt/tweets/" + os.path.splitext(f)[0] + "-parsed.csv")

Processing 0 - 80000


Process ForkPoolWorker-290:
Process ForkPoolWorker-295:
Process ForkPoolWorker-292:
Process ForkPoolWorker-294:
Process ForkPoolWorker-293:
Process ForkPoolWorker-296:
Process ForkPoolWorker-291:
Process ForkPoolWorker-289:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
Traceback (most recent call last):
TypeError: getresponse() got an unexpected keyword argument 'buffering'
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/

KeyboardInterrupt: 

  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-167-15fabf5806ba>", line 19, in get_statuses
    return np.asarray(api.GetStatuses(df['t_id']))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/local/lib/python3.6/dist-packages/twitter/api.py", line 902, in GetStatuses
    resp = self._RequestUrl(url, 'GET', data=parameters)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))

During handling of the above exception, another exception occurred:

  File "/usr/lib/python3.6/multiprocessing/process.py", line

  File "/usr/local/lib/python3.6/dist-packages/requests/sessions.py", line 508, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python3.6/dist-packages/requests/sessions.py", line 618, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/requests/adapters.py", line 440, in send
    timeout=timeout
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 850, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.6/dist-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwarg

  File "/usr/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/soc